In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
stat1 = pd.read_csv(r"C:\Users\ASUS\Documents\WIP\shotlog\2023stat.csv")
stat1 = stat1[['PLAYER_NAME','TEAM_NAME','GAME_ID','SHOT_MADE','QUARTER','MINS_LEFT','SECS_LEFT']]
stat1 = stat1.rename(columns={'PLAYER_NAME':'player','TEAM_NAME':'team'})

stat2 = pd.read_csv(r"C:\Users\ASUS\Documents\WIP\shotlog\player_stat_2023.csv",encoding='latin-1')
stat2 = stat2.rename(columns={'ï»¿Player': 'player','Tm': 'team'})

In [4]:
team_abbreviations = {
    'Atlanta Hawks': 'ATL','Boston Celtics': 'BOS','Brooklyn Nets': 'BKN','Charlotte Hornets': 'CHA','Chicago Bulls': 'CHI','Cleveland Cavaliers': 'CLE', 'Dallas Mavericks': 'DAL','Denver Nuggets': 'DEN', 'Detroit Pistons': 'DET','Golden State Warriors': 'GSW','Houston Rockets': 'HOU','Indiana Pacers': 'IND','LA Clippers': 'LAC','Los Angeles Lakers': 'LAL','Memphis Grizzlies': 'MEM','Miami Heat': 'MIA','Milwaukee Bucks': 'MIL','Minnesota Timberwolves': 'MIN','New Orleans Pelicans': 'NOP','New York Knicks': 'NYK','Oklahoma City Thunder': 'OKC','Orlando Magic': 'ORL','Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHX','Portland Trail Blazers': 'POR','Sacramento Kings': 'SAC','San Antonio Spurs': 'SAS','Toronto Raptors': 'TOR','Utah Jazz': 'UTA','Washington Wizards': 'WAS'}
stat1['team'] = stat1['team'].replace(team_abbreviations)

In [5]:
stat1['current_shot_hit'] = np.where(stat1['SHOT_MADE']==True, 1, 0)
stat1['lag_shot_hit']=stat1.groupby(['player','GAME_ID'])['current_shot_hit'].shift(1)

In [6]:
shotlog = pd.merge(stat1,stat2,on=['player','team'],how='inner')
shotlog=shotlog[pd.notnull(shotlog["lag_shot_hit"])]
shotlog['conse_shot_hit'] = np.where((shotlog['current_shot_hit']==1)&(shotlog['lag_shot_hit']==1), 1, 0) 

In [7]:
shotlog

,player,team,GAME_ID,SHOT_MADE,QUARTER,MINS_LEFT,SECS_LEFT,current_shot_hit,lag_shot_hit,FG%,conse_shot_hit
1,Bradley Beal,WAS,22200004,False,1,5,28,0,1.0,0.506,0
2,Bradley Beal,WAS,22200004,True,1,5,24,1,0.0,0.506,0
3,Bradley Beal,WAS,22200004,True,1,4,3,1,1.0,0.506,1
4,Bradley Beal,WAS,22200004,False,1,2,44,0,1.0,0.506,0
5,Bradley Beal,WAS,22200004,True,2,4,31,1,0.0,0.506,0
...,...,...,...,...,...,...,...,...,...,...,...
181906,Bruno Fernando,ATL,22201216,True,4,1,2,1,1.0,0.579,1
181907,Bruno Fernando,ATL,22201216,False,4,0,28,0,1.0,0.579,0
181910,Donovan Williams,ATL,22201216,True,4,1,37,1,0.0,0.400,0
181911,Donovan Williams,ATL,22201216,False,4,1,12,0,1.0,0.400,0


In [8]:
Player_Prob=shotlog.groupby(['player'])[['conse_shot_hit','lag_shot_hit']].mean()
Player_Prob=Player_Prob.reset_index()
Player_Prob.rename(columns={'lag_shot_hit':'avg_lag_hit'}, inplace=True)
Player_Prob.rename(columns={'conse_shot_hit':'avg_conse_hit'}, inplace=True)
Player_Prob['conditional_prob']=Player_Prob['avg_conse_hit']/Player_Prob['avg_lag_hit']

In [9]:
avg = shotlog.groupby(['player'])['FG%'].mean()
Player_stat = pd.merge(Player_Prob,avg, on='player',how='left')
Player_stat=Player_stat[pd.notnull(Player_stat["conditional_prob"])]

shot_taken = shotlog.groupby(['player'])['SHOT_MADE'].count()
Player_stat=pd.merge(Player_stat,shot_taken,on='player').rename(columns={'SHOT_MADE': 'shot_taken'})

In [10]:
Player_stat['difference'] = Player_stat['conditional_prob']-Player_stat['FG%']
Player_stat = Player_stat[Player_stat['shot_taken']>200]
Player_stat.sort_values('difference',ascending=False)

,player,avg_conse_hit,avg_lag_hit,conditional_prob,FG%,shot_taken,difference
44,Cade Cunningham,0.207547,0.429245,0.483516,0.415,212,0.068516
219,Jonathan Kuminga,0.317865,0.538283,0.590517,0.525,431,0.065517
60,Clint Capela,0.460722,0.651805,0.706840,0.653,471,0.053840
25,Ayo Dosunmu,0.272912,0.507128,0.538153,0.493,491,0.045153
401,Tari Eason,0.221453,0.449827,0.492308,0.448,578,0.044308
...,...,...,...,...,...,...,...
396,Steven Adams,0.298643,0.565611,0.528000,0.597,221,-0.069000
391,Simone Fontecchio,0.113208,0.377358,0.300000,0.369,265,-0.069000
255,Kenrich Williams,0.242321,0.542662,0.446541,0.517,293,-0.070459
107,Dorian Finney-Smith,0.140741,0.418519,0.336283,0.416,270,-0.079717


In [11]:
import scipy.stats as sp
sp.stats.ttest_rel(Player_stat['conditional_prob'], Player_stat['FG%'],alternative='greater')

TtestResult(statistic=-5.717387991366644, pvalue=0.9999999842638764, df=244)

In [12]:
sp.stats.ttest_rel(Player_stat['conditional_prob'], Player_stat['FG%'],alternative='less')

TtestResult(statistic=-5.717387991366644, pvalue=1.5736123633759192e-08, df=244)

In [13]:
Autocorr_Hit=shotlog.groupby('player')[['current_shot_hit','lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit.columns=Autocorr_Hit.columns.get_level_values(0)
Autocorr_Hit.rename(columns={'current_shot_hit':'autocorr'}, inplace=True)

shot_taken = shotlog.groupby(['player'])['SHOT_MADE'].count()
Autocorr_Hit=pd.merge(Autocorr_Hit,shot_taken,on='player').rename(columns={'SHOT_MADE': 'shot_taken'})

Autocorr_Hit = Autocorr_Hit[pd.notnull(Autocorr_Hit['autocorr']) ]
Autocorr_Hit = Autocorr_Hit[Autocorr_Hit['shot_taken']>200]
Autocorr_Hit.sort_values('autocorr',ascending=False)

,player,autocorr,shot_taken
222,Jonathan Kuminga,0.123098,431
60,Clint Capela,0.122568,471
44,Cade Cunningham,0.111941,212
14,Andre Drummond,0.101030,212
111,Drew Eubanks,0.092419,259
...,...,...,...
38,Bruce Brown,-0.108321,661
108,Dorian Finney-Smith,-0.110341,270
206,Jeff Green,-0.131583,272
143,Hamidou Diallo,-0.135417,319


In [14]:
sp.stats.ttest_1samp(Autocorr_Hit['autocorr'], 0)

TtestResult(statistic=-6.310936057264836, pvalue=1.2967378540982061e-09, df=244)